In [ ]:
#1. Install the ultralytics package
#!pip install ultralytics
!pip install -U ultralytics

In [ ]:
!pip install openpyxl

In [ ]:

# 3. สร้างไฟล์ data.yaml สำหรับการ train YOLO
data_yaml = """
# ❗ เปลี่ยน path ให้เป็น path จริงบนเครื่องของคุณ
path: C:\\Users\\Wongpanya.Nu\\DB_BAN\\Dataset
train: images/train
val: images/val
test: images/test

names:
    0: Candyapple
    1: Namwa
    2: Namwadam
    3: Homthon
    4: Nak
"""

with open("data.yaml", "w", encoding="utf-8") as f:
        f.write(data_yaml)

In [8]:
# 4.เช็กและลบ label ผิดก่อน train:
import os

def clean_labels(image_dir, label_dir):
    removed = 0
    # Check if label_dir exists before listing its contents
    if not os.path.exists(label_dir):
        print(f"Warning: Label directory not found at {label_dir}. Skipping cleaning for this set.")
        return

    for file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, file)
        # Assuming images are .jpg or .png
        image_path_jpg = os.path.join(image_dir, file.replace('.txt', '.jpg'))
        image_path_png = os.path.join(image_dir, file.replace('.txt', '.png'))

        if not os.path.exists(image_path_jpg) and not os.path.exists(image_path_png):
            try:
                os.remove(label_path)
                removed += 1
            except OSError as e:
                print(f"Error deleting file {label_path}: {e}")

    print(f"✅ ลบ label ที่ไม่มีภาพออกแล้ว: {removed} ไฟล์ ใน {label_dir}")

# เรียกใช้กับทุกชุด โดยใช้พาธที่ถูกต้อง
base_dir = r'C:\skin_project\herbSkin_Project66\NEW\dataset'
clean_labels(os.path.join(base_dir, 'images', 'train'), os.path.join(base_dir, 'labels', 'train'))
clean_labels(os.path.join(base_dir, 'images', 'val'), os.path.join(base_dir, 'labels', 'val'))
clean_labels(os.path.join(base_dir, 'images', 'test'), os.path.join(base_dir, 'labels', 'test'))

✅ ลบ label ที่ไม่มีภาพออกแล้ว: 0 ไฟล์ ใน C:\skin_project\herbSkin_Project66\NEW\dataset\labels\train
✅ ลบ label ที่ไม่มีภาพออกแล้ว: 0 ไฟล์ ใน C:\skin_project\herbSkin_Project66\NEW\dataset\labels\val


In [12]:
# 5.อ่านและแสดงเนื้อหาของไฟล์ data.yaml
# ไฟล์นี้ใช้กำหนดการตั้งค่า dataset สำหรับการฝึกโมเดล YOLO

yaml_path = r'C:\skin_project\herbSkin_Project66\NEW\data.yml'
try:
    with open(yaml_path, 'r', encoding='utf-8') as f:
        yaml_content = f.read()
    print("Content of data.yaml:")
    print(yaml_content)
except FileNotFoundError:
    print(f"Error: {yaml_path} not found. Please ensure the data.yaml file exists in the specified location.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

Content of data.yaml:
names:
  0: Snake Plant
  1: Turmeric
  2: Galanga
  3: cucumber
  4: Alovera
  5: Garlic
  6: Houttuynia cordata
  7: pluLeaf
  8: Ivy Gourd
  9: Mangosteen Peel
  10: khaproa
  11: horapa
nc: 12
path: C:\skin_project\herbSkin_Project66\new\dataset
test: images/test
train: images/train
val: images/val



In [15]:
# 6. Train YOLOv8 Model and visualize results
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch

# 1. Train YOLOv8
model = YOLO('yolov8n.pt')
results = model.train(
    data=r'C:\skin_project\herbSkin_Project66\NEW\data.yml',
    epochs=100,
    imgsz=640,
    augment=True,
    degrees=1,
    scale=0.5,
    flipud=0.5,
    fliplr=0.5,
    batch=16,
    device='cuda:0' if torch.cuda.is_available() else 'cpu',
    optimizer='AdamW'
)
print("✅ การฝึกโมเดลเสร็จสมบูรณ์แล้ว!")

# 2. Convert results.csv to results.xlsx if not exist
csv_path = r'runs\detect\train\results.csv'
excel_path = r'runs\detect\train\results.xlsx'
if not os.path.exists(excel_path) and os.path.exists(csv_path):
    df_csv = pd.read_csv(csv_path)
    df_csv.to_excel(excel_path, index=False)
    print("✅ แปลง results.csv เป็น results.xlsx เรียบร้อยแล้ว")

# 3. Read and plot metrics from results.xlsx
if os.path.exists(excel_path):
    df = pd.read_excel(excel_path)
    print("Columns:", df.columns.tolist())

    metrics_to_plot = {
        'metrics/mAP_0.5:0.95': 'mAP50-95 vs. Epoch',
        'metrics/mAP_0.5': 'mAP50 vs. Epoch',
        'metrics/precision': 'Precision vs. Epoch',
        'metrics/recall': 'Recall vs. Epoch'
    }

    for metric_col, title in metrics_to_plot.items():
        if metric_col in df.columns:
            plt.figure(figsize=(10, 6))
            plt.plot(df.index, df[metric_col])
            plt.xlabel('Epoch')
            plt.ylabel(metric_col.split('/')[-1])
            plt.title(title)
            plt.grid(True)
            plt.show()
        else:
            print(f"Warning: Metric column '{metric_col}' not found in results.xlsx.")
else:
    print(f"Error: ไม่พบไฟล์ {excel_path}")

Ultralytics 8.3.184  Python-3.13.3 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\skin_project\herbSkin_Project66\NEW\data.yml, degrees=1, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plot

train: Scanning C:\skin_project\herbSkin_Project66\new\dataset\labels\train.cache... 7200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7200/7200 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 1519.5819.5 MB/s, size: 255.7 KB)


val: Scanning C:\skin_project\herbSkin_Project66\new\dataset\labels\val.cache... 4800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4800/4800 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4800       5299      0.988       0.98      0.989      0.978

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.54G     0.1984     0.1585     0.8385         17        640: 100%|██████████| 450/450 [00:24<00:00, 18.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:13<00:00, 11.32it/s]


                   all       4800       5299      0.988       0.98      0.989      0.978

100 epochs completed in 1.061 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.3MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics 8.3.184  Python-3.13.3 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:19<00:00,  7.56it/s]


                   all       4800       5299       0.99       0.98      0.989      0.978
           Snake Plant        390        414          1      0.949      0.965      0.951
              Turmeric        420        445      0.998      0.962      0.994      0.978
               Galanga        396        404      0.982          1      0.995      0.992
              cucumber        415        434      0.998      0.977      0.995      0.979
               Alovera        372        374      0.994      0.995      0.995      0.987
                Garlic        377        668      0.991      0.997      0.995      0.987
    Houttuynia cordata        422        432      0.988      0.993      0.995      0.988
               pluLeaf        401        403          1      0.996      0.995      0.993
             Ivy Gourd        404        405      0.992      0.998      0.995      0.995
       Mangosteen Peel        397        471       0.99      0.985      0.995      0.976
               khapro

In [16]:
# 2. Convert results.csv to results.xlsx if not exist
csv_path = r'runs\detect\train2\results.csv'
excel_path = r'runs\detect\train2\results.xlsx'
if not os.path.exists(excel_path) and os.path.exists(csv_path):
    df_csv = pd.read_csv(csv_path)
    df_csv.to_excel(excel_path, index=False)
    print("✅ แปลง results.csv เป็น results.xlsx เรียบร้อยแล้ว")

# 3. Read and plot metrics from results.xlsx
if os.path.exists(excel_path):
    df = pd.read_excel(excel_path)
    print("Columns:", df.columns.tolist())

    metrics_to_plot = {
        'metrics/mAP_0.5:0.95': 'mAP50-95 vs. Epoch',
        'metrics/mAP_0.5': 'mAP50 vs. Epoch',
        'metrics/precision': 'Precision vs. Epoch',
        'metrics/recall': 'Recall vs. Epoch'
    }

    for metric_col, title in metrics_to_plot.items():
        if metric_col in df.columns:
            plt.figure(figsize=(10, 6))
            plt.plot(df.index, df[metric_col])
            plt.xlabel('Epoch')
            plt.ylabel(metric_col.split('/')[-1])
            plt.title(title)
            plt.grid(True)
            plt.show()
        else:
            print(f"Warning: Metric column '{metric_col}' not found in results.xlsx.")
else:
    print(f"Error: ไม่พบไฟล์ {excel_path}")

✅ แปลง results.csv เป็น results.xlsx เรียบร้อยแล้ว
Columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']


In [ ]:
#สร้างกราฟ ของ validation และ training
import pandas as pd
import matplotlib.pyplot as plt

def plot_training_results(results_path, save_path=None):
    """พลอตกราฟผลการ training และ validation แบบ 10 กราฟตามตาราง"""
    
    try:
        df = pd.read_csv(results_path)
        print(f"โหลดข้อมูลสำเร็จ: {df.shape[0]} epochs")
        print(f"คอลัมน์ที่มี: {list(df.columns)}")
    except FileNotFoundError:
        print(f"ไม่พบไฟล์ {results_path}")
        return
    
    # สร้าง figure แบบ 2x5 = 10 กราฟ
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('Training and Validation Results', fontsize=16, fontweight='bold')
    
    epochs = range(1, len(df) + 1)
    
    # แถวที่ 1: Training losses และ metrics
    # 1. Box Loss
    if 'train/box_loss' in df.columns:
        axes[0, 0].plot(epochs, df['train/box_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 0].set_title('train/box_loss')
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Class Loss
    if 'train/cls_loss' in df.columns:
        axes[0, 1].plot(epochs, df['train/cls_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 1].set_title('train/cls_loss')
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. DFL Loss
    if 'train/dfl_loss' in df.columns:
        axes[0, 2].plot(epochs, df['train/dfl_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 2].set_title('train/dfl_loss')
        axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Precision
    if 'metrics/precision(B)' in df.columns:
        axes[0, 3].plot(epochs, df['metrics/precision(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 3].set_title('metrics/precision(B)')
        axes[0, 3].grid(True, alpha=0.3)
    
    # 5. Recall
    if 'metrics/recall(B)' in df.columns:
        axes[0, 4].plot(epochs, df['metrics/recall(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 4].set_title('metrics/recall(B)')
        axes[0, 4].grid(True, alpha=0.3)
    
    # แถวที่ 2: Validation losses และ mAP
    # 6. Val Box Loss
    if 'val/box_loss' in df.columns:
        axes[1, 0].plot(epochs, df['val/box_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 0].set_title('val/box_loss')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 7. Val Class Loss
    if 'val/cls_loss' in df.columns:
        axes[1, 1].plot(epochs, df['val/cls_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 1].set_title('val/cls_loss')
        axes[1, 1].grid(True, alpha=0.3)
    
    # 8. Val DFL Loss
    if 'val/dfl_loss' in df.columns:
        axes[1, 2].plot(epochs, df['val/dfl_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 2].set_title('val/dfl_loss')
        axes[1, 2].grid(True, alpha=0.3)
    
    # 9. mAP50
    if 'metrics/mAP50(B)' in df.columns:
        axes[1, 3].plot(epochs, df['metrics/mAP50(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 3].set_title('metrics/mAP50(B)')
        axes[1, 3].grid(True, alpha=0.3)
    
    # 10. mAP50-95
    if 'metrics/mAP50-95(B)' in df.columns:
        axes[1, 4].plot(epochs, df['metrics/mAP50-95(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 4].set_title('metrics/mAP50-95(B)')
        axes[1, 4].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"บันทึกกราฟที่: {save_path}")
    
    plt.show()

def find_best_epoch(results_path, metric='metrics/mAP50(B)'):
    """หา epoch ที่ดีที่สุด"""
    df = pd.read_csv(results_path)
    
    if metric in df.columns:
        best_epoch = df[metric].idxmax() + 1
        best_value = df[metric].max()
        print(f"Best {metric}: {best_value:.4f} at epoch {best_epoch}")
        return best_epoch, best_value
    else:
        print(f"ไม่พบ metric: {metric}")

# วิธีการใช้งาน
if __name__ == "__main__":
    
    # แก้ path ตรงนี้
    RESULTS_PATH = r'C:\skin_project\herbSkin_Project66\new\runs\detect\train2\results.csv'
    
    # พลอตกราฟ 10 แบบ
    plot_training_results(RESULTS_PATH, 'training_results.png')
    
    # หา epoch ที่ดีที่สุด
    find_best_epoch(RESULTS_PATH)

In [ ]:
# title แก้ไม่ได้แก้ไม่เป็นพยายามแล้ว
# 7.Evaluate the model on the test set and save metrics to Excel and show confusion matrix
#
import pandas as pd
import numpy as np
from ultralytics import YOLO
import torch
import os
from PIL import Image
import matplotlib.pyplot as plt

# 1. Evaluate the model on the test set
model = YOLO(r'runs\detect\train2\weights\best.pt')  # ปรับ path ให้ตรงกับ best.pt ล่าสุด
metrics = model.val(split='test', data=r'C:\skin_project\herbSkin_Project66\new\data.yml')
print(metrics)

# 2. Extract relevant metrics
metrics_data = {
    'Metric': ['mAP50-95', 'mAP50', 'Precision', 'Recall'],
    'Overall': [
        metrics.box.maps.mean() if hasattr(metrics.box, 'maps') else np.nan,
        metrics.box.map50 if hasattr(metrics.box, 'map50') else np.nan,
        metrics.box.mp if hasattr(metrics.box, 'mp') else np.nan,
        metrics.box.mr if hasattr(metrics.box, 'mr') else np.nan
    ]
}

# Per-class metrics
num_classes = len(metrics.names)
ap_per_class = metrics.box.ap if hasattr(metrics.box, 'ap') and isinstance(metrics.box.ap, np.ndarray) and len(metrics.box.ap) == num_classes else [np.nan] * num_classes
ap50_per_class = metrics.box.ap50 if hasattr(metrics.box, 'ap50') and isinstance(metrics.box.ap50, np.ndarray) and len(metrics.box.ap50) == num_classes else [np.nan] * num_classes
p_per_class = metrics.box.p if hasattr(metrics.box, 'p') and isinstance(metrics.box.p, np.ndarray) and len(metrics.box.p) == num_classes else [np.nan] * num_classes
r_per_class = metrics.box.r if hasattr(metrics.box, 'r') and isinstance(metrics.box.r, np.ndarray) and len(metrics.box.r) == num_classes else [np.nan] * num_classes

for i, class_name in enumerate(metrics.names):
    metrics_data[class_name] = [ap_per_class[i], ap50_per_class[i], p_per_class[i], r_per_class[i]]

# 3. Save metrics to Excel
metrics_df = pd.DataFrame(metrics_data)
excel_output_path = r'C:\skin_project\herbSkin_Project66\new\evaluation_metrics.xlsx'
metrics_df.to_excel(excel_output_path, index=False)
print(f"✅ Evaluation metrics saved to {excel_output_path}")

# 4. Show confusion matrix
conf_matrix_path = r'runs\detect\val\confusion_matrix.png'
if os.path.exists(conf_matrix_path):
    img = Image.open(conf_matrix_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title("Confusion Matrix")
    plt.show()
else:
    print(f"ไม่พบไฟล์ {conf_matrix_path}")

# 5. Predict and show results on test images
results = model.predict(
    source=r'C:\skin_project\herbSkin_Project66\new\dataset\images\test',
    conf=0.5,
    save=True  # จะบันทึกภาพที่ตรวจจับแล้วไว้ใน runs/
)

# 6. Save predict log to Excel
log_data = []
for result in results:
    img_path = result.path if hasattr(result, 'path') else None
    for box, conf, cls in zip(result.boxes.xyxy.cpu().numpy(),
                              result.boxes.conf.cpu().numpy(),
                              result.boxes.cls.cpu().numpy()):
        log_data.append({
            'image': img_path,
            'class': result.names[int(cls)],
            'confidence': float(conf),
            'x1': float(box[0]),
            'y1': float(box[1]),
            'x2': float(box[2]),
            'y2': float(box[3])
        })
df_log = pd.DataFrame(log_data)
log_path = r'C:\skin_project\herbSkin_Project66\new\predict_log.xlsx'
df_log.to_excel(log_path, index=False)
print(f"✅ Predict log saved to {log_path}")

# 7. Show example prediction images
for result in results[:3]:  # แสดงแค่ 3 ภาพตัวอย่าง
    img = result.plot()  # วาดกรอบผลลัพธ์ลงบนภาพ
    plt.figure(figsize=(10, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Prediction Result")
    plt.show()

Ultralytics 8.3.184  Python-3.13.3 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5080, 16303MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1202.2746.4 MB/s, size: 212.3 KB)


val: Scanning C:\skin_project\herbSkin_Project66\new\dataset\labels\test.cache... 0 images, 18 backgrounds, 0 corrupt: 100%|██████████| 18/18 [00:00<?, ?it/s]

WARNING Labels are missing or empty in C:\skin_project\herbSkin_Project66\new\dataset\labels\test.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\ultralytics\utils\metrics.py:668: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\ultralytics\utils\metrics.py:713: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packa

                   all         18          0          0          0          0          0
WARNING no labels found in detect set, can not compute metrics without labels
Speed: 2.7ms preprocess, 12.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs\detect\val6
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([], dtype=int64)
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000201F349ED60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.0

<Figure size 640x480 with 1 Axes>


image 1/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Alovera_002-no-bg.png: 640x640 1 Alovera, 2.9ms
image 2/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Alovera_004-no-bg.png: 640x640 1 Alovera, 3.0ms
image 3/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Alovera_005-no-bg.png: 640x640 1 Alovera, 3.0ms
image 4/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Galanga_001.png: 640x640 1 Galanga, 2.9ms
image 5/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Galanga_003.png: 640x640 1 Galanga, 2.7ms
image 6/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Galanga_005.png: 640x640 1 Galanga, 3.1ms
image 7/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Garlic_001-no-bg.png: 640x640 1 Garlic, 3.1ms
image 8/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Garlic_002-no-bg.png: 640x640 1 Garlic, 2.7ms
image 9/18 C:\skin_project\herbSkin_Project66\new\dataset\images\test\Garlic_

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

<Figure size 1000x600 with 1 Axes>

In [22]:
#export YOLOv8 เป็น ONNX และ TensorFlow SavedMode
# 8. Export YOLOv8 model to ONNX and TensorFlow SavedModel
# 1. ติดตั้ง tensorflowjs
#!pip install tensorflowjs
## 2. คำสั่งนี้ใน terminal: tensorflowjs_converter --input_format=tf_saved_model runs\detect\train\weights\best_saved_model ./best_tfjs

from ultralytics import YOLO

# โหลดโมเดลที่ train เสร็จแล้ว
model = YOLO(r'runs\detect\train2\weights\best.pt')

# Export เป็น ONNX
model.export(format='onnx', dynamic=True, opset=12)
# จะได้ไฟล์ runs\detect\train\weights\best.onnx

# Export เป็น TensorFlow SavedModel
model.export(format='tf')
# จะได้โฟลเดอร์ runs\detect\train\weights\best_saved_model/


Ultralytics 8.3.184  Python-3.13.3 torch-2.8.0+cu129 CPU (AMD Ryzen 7 9800X3D 8-Core Processor)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train2\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.59" "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.59" "onnxruntime-gpu" ' returned non-zero exit status 1.
WARNING requirements:  Command 'pip install --no-cache-dir "onnx>=1.12.0,<1.18.0" "onnxslim>=0.1.59" "onnxruntime-gpu" ' returned non-zero exit status 1.
ERROR ONNX: export failure 97.5s: No module named 'onnx'


ModuleNotFoundError: No module named 'onnx'

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001FED02902C0>
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py", line 1622, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [32]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_training_results(results_path, save_path=None):
    """พลอตกราฟผลการ training และ validation แบบ 10 กราฟตามตาราง"""
    
    try:
        df = pd.read_csv(results_path)
        print(f"โหลดข้อมูลสำเร็จ: {df.shape[0]} epochs")
        print(f"คอลัมน์ที่มี: {list(df.columns)}")
    except FileNotFoundError:
        print(f"ไม่พบไฟล์ {results_path}")
        return
    
    # สร้าง figure แบบ 2x5 = 10 กราฟ
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('Training and Validation Results', fontsize=16, fontweight='bold')
    
    epochs = range(1, len(df) + 1)
    
    # แถวที่ 1: Training losses และ metrics
    # 1. Box Loss
    if 'train/box_loss' in df.columns:
        axes[0, 0].plot(epochs, df['train/box_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 0].set_title('train/box_loss')
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Class Loss
    if 'train/cls_loss' in df.columns:
        axes[0, 1].plot(epochs, df['train/cls_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 1].set_title('train/cls_loss')
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. DFL Loss
    if 'train/dfl_loss' in df.columns:
        axes[0, 2].plot(epochs, df['train/dfl_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 2].set_title('train/dfl_loss')
        axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Precision
    if 'metrics/precision(B)' in df.columns:
        axes[0, 3].plot(epochs, df['metrics/precision(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 3].set_title('metrics/precision(B)')
        axes[0, 3].grid(True, alpha=0.3)
    
    # 5. Recall
    if 'metrics/recall(B)' in df.columns:
        axes[0, 4].plot(epochs, df['metrics/recall(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[0, 4].set_title('metrics/recall(B)')
        axes[0, 4].grid(True, alpha=0.3)
    
    # แถวที่ 2: Validation losses และ mAP
    # 6. Val Box Loss
    if 'val/box_loss' in df.columns:
        axes[1, 0].plot(epochs, df['val/box_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 0].set_title('val/box_loss')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 7. Val Class Loss
    if 'val/cls_loss' in df.columns:
        axes[1, 1].plot(epochs, df['val/cls_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 1].set_title('val/cls_loss')
        axes[1, 1].grid(True, alpha=0.3)
    
    # 8. Val DFL Loss
    if 'val/dfl_loss' in df.columns:
        axes[1, 2].plot(epochs, df['val/dfl_loss'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 2].set_title('val/dfl_loss')
        axes[1, 2].grid(True, alpha=0.3)
    
    # 9. mAP50
    if 'metrics/mAP50(B)' in df.columns:
        axes[1, 3].plot(epochs, df['metrics/mAP50(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 3].set_title('metrics/mAP50(B)')
        axes[1, 3].grid(True, alpha=0.3)
    
    # 10. mAP50-95
    if 'metrics/mAP50-95(B)' in df.columns:
        axes[1, 4].plot(epochs, df['metrics/mAP50-95(B)'], 'o-', label='results', linewidth=2, markersize=3)
        axes[1, 4].set_title('metrics/mAP50-95(B)')
        axes[1, 4].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"บันทึกกราฟที่: {save_path}")
    
    plt.show()

def find_best_epoch(results_path, metric='metrics/mAP50(B)'):
    """หา epoch ที่ดีที่สุด"""
    df = pd.read_csv(results_path)
    
    if metric in df.columns:
        best_epoch = df[metric].idxmax() + 1
        best_value = df[metric].max()
        print(f"Best {metric}: {best_value:.4f} at epoch {best_epoch}")
        return best_epoch, best_value
    else:
        print(f"ไม่พบ metric: {metric}")

# วิธีการใช้งาน
if __name__ == "__main__":
    
    # แก้ path ตรงนี้
    RESULTS_PATH = r'C:\skin_project\herbSkin_Project66\new\runs\detect\train2\results.csv'
    
    # พลอตกราฟ 10 แบบ
    plot_training_results(RESULTS_PATH, 'training_results.png')
    
    # หา epoch ที่ดีที่สุด
    find_best_epoch(RESULTS_PATH)

โหลดข้อมูลสำเร็จ: 100 epochs
คอลัมน์ที่มี: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']
บันทึกกราฟที่: training_results.png


<Figure size 2000x800 with 10 Axes>

Best metrics/mAP50(B): 0.9890 at epoch 99


In [ ]:
# โดดข้ามเลยจ้า เครื่องไม่แรงพอแน่ๆ
# 8. Compare YOLO versions and visualize results
#
#
#
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
import os

model_versions = [
    {'name': 'YOLOv8', 'weight': 'yolov8n.pt'},
    {'name': 'YOLOv9', 'weight': 'yolov9n.pt'},
    {'name': 'YOLOv10', 'weight': 'yolov10n.pt'},
    {'name': 'YOLOv11', 'weight': 'yolov11n.pt'},
]

results_summary = []

for model_info in model_versions:
    print(f"=== Training {model_info['name']} ===")
    model = YOLO(model_info['weight'])
    results = model.train(
        data=r'C:\Users\Wongpanya.Nu\DB_BAN\data.yaml',
        epochs=1,
        imgsz=640,
        batch=16,
        device='cuda:0',
        optimizer='AdamW',
        project=f'results_{model_info["name"]}',  # แยกโฟลเดอร์
        name='exp'
    )
    # Evaluate
    metrics = model.val(split='test', data=r'C:\Users\Wongpanya.Nu\DB_BAN\data.yaml')
    # เก็บ summary
    results_summary.append({
        'Model': model_info['name'],
        'mAP50-95': metrics.box.maps.mean() if hasattr(metrics.box, 'maps') else None,
        'mAP50': metrics.box.map50 if hasattr(metrics.box, 'map50') else None,
        'Precision': metrics.box.mp if hasattr(metrics.box, 'mp') else None,
        'Recall': metrics.box.mr if hasattr(metrics.box, 'mr') else None
    })
    # สามารถ save confusion matrix, log, ฯลฯ เพิ่มเติมได้

# สร้าง DataFrame เปรียบเทียบ
df_compare = pd.DataFrame(results_summary)
print(df_compare)

# Plot เปรียบเทียบ
plt.figure(figsize=(10,6))
plt.bar(df_compare['Model'], df_compare['mAP50-95'])
plt.ylabel('mAP50-95')
plt.title('Comparison of mAP50-95 between YOLO versions')
plt.show()

# Export summary to Excel
df_compare.to_excel('compare_yolo_versions.xlsx', index=False)

In [ ]:
# Structure of results directory
""" DB_BAN/
│
├─ results_yolov8/
│    ├─ train/...
│    ├─ val/...
│    ├─ metrics.csv
│    ├─ predict_log.xlsx
│    └─ confusion_matrix.png
├─ results_yolov9/
│    ├─ train/...
│    ├─ val/...
│    ├─ metrics.csv
│    ├─ predict_log.xlsx
│    └─ confusion_matrix.png
...
"

In [ ]:
"""
โปรดอ่านกให้เข้าใจก่อนเริ่มใช้งาน
# 1. ติดตั้ง ultralytics และ openpyxl  
# 2. สร้างไฟล์ data.yaml สำหรับการ train YOLO
# 3. เช็กและลบ label ผิดก่อน train
# 4. อ่านและแสดงเนื้อหาของไฟล์ data.yaml
# 5. Train YOLOv8 Model and visualize results
# 6. Evaluate the model on the test set and save metrics to Excel and show confusion matrix
# 7. Predict and show results on test images
# 8. Compare YOLO versions and visualize results  โดดข้ามเลนช้าเครื่องช้า ให้ไปทำ 9-10
# 9. Export YOLOv8-v11 ทีละครั้ง model 
# 10. Export model to ONNX and TensorFlow SavedModel
"""

In [ ]:
#1. Train, Evaluate, Predict, Log (รันทีละโมเดล) YOLOv8-v11 
from ultralytics import YOLO
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt

# ตั้งชื่อเวอร์ชันและโฟลเดอร์ผลลัพธ์
model_name = 'yolov8n.pt'  # เปลี่ยนเป็น yolov9n.pt, ...
result_dir = 'results_yolov8'  # เปลี่ยนตามเวอร์ชัน
os.makedirs(result_dir, exist_ok=True)

# 1. Train
model = YOLO(model_name)
results = model.train(
    data=r'C:\Users\Wongpanya.Nu\DB_BAN\data.yaml',
    epochs=2,
    imgsz=640,
    batch=16,
    device='cuda:0',
    optimizer='AdamW',
    project=result_dir,
    name='train'
)

# 2. Evaluate
metrics = model.val(split='test', data=r'C:\Users\Wongpanya.Nu\DB_BAN\data.yaml', project=result_dir, name='val')

# 3. Save metrics summary
metrics_data = {
    'Metric': ['mAP50-95', 'mAP50', 'Precision', 'Recall'],
    'Overall': [
        metrics.box.maps.mean() if hasattr(metrics.box, 'maps') else np.nan,
        metrics.box.map50 if hasattr(metrics.box, 'map50') else np.nan,
        metrics.box.mp if hasattr(metrics.box, 'mp') else np.nan,
        metrics.box.mr if hasattr(metrics.box, 'mr') else np.nan
    ]
}
metrics_df = pd.DataFrame(metrics_data)
metrics_df.to_csv(os.path.join(result_dir, 'metrics.csv'), index=False)

# 4. Predict & save log
results = model.predict(
    source=r'C:\Users\Wongpanya.Nu\DB_BAN\Dataset\images\test',
    conf=0.5,
    save=True,
    project=result_dir,
    name='predict'
)
log_data = []
for result in results:
    img_path = result.path if hasattr(result, 'path') else None
    for box, conf, cls in zip(result.boxes.xyxy.cpu().numpy(),
                              result.boxes.conf.cpu().numpy(),
                              result.boxes.cls.cpu().numpy()):
        log_data.append({
            'image': img_path,
            'class': result.names[int(cls)],
            'confidence': float(conf),
            'x1': float(box[0]),
            'y1': float(box[1]),
            'x2': float(box[2]),
            'y2': float(box[3])
        })
df_log = pd.DataFrame(log_data)
df_log.to_excel(os.path.join(result_dir, 'predict_log.xlsx'), index=False)

# 5. Copy confusion matrix
conf_matrix_path = os.path.join(result_dir, 'val', 'confusion_matrix.png')
if os.path.exists(conf_matrix_path):
    # สามารถ copy หรือ rename ได้ตามต้องการ
    pass

In [ ]:
"""
2. หลังจากรันครบทุกโมเดลแล้ว
นำ metrics.csv, predict_log.xlsx, confusion_matrix.png ของแต่ละโฟลเดอร์มาเปรียบเทียบ

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

model_dirs = ['results_yolov8', 'results_yolov9', 'results_yolov10', 'results_yolov11']
summary = []

for d in model_dirs:
    metrics_path = os.path.join(d, 'metrics.csv')
    if os.path.exists(metrics_path):
        df = pd.read_csv(metrics_path)
        summary.append({
            'Model': d,
            'mAP50-95': df.loc[df['Metric'] == 'mAP50-95', 'Overall'].values[0],
            'mAP50': df.loc[df['Metric'] == 'mAP50', 'Overall'].values[0],
            'Precision': df.loc[df['Metric'] == 'Precision', 'Overall'].values[0],
            'Recall': df.loc[df['Metric'] == 'Recall', 'Overall'].values[0],
        })

df_compare = pd.DataFrame(summary)
print(df_compare)

# Plot
plt.figure(figsize=(10,6))
plt.bar(df_compare['Model'], df_compare['mAP50-95'])
plt.ylabel('mAP50-95')
plt.title('Comparison of mAP50-95 between YOLO versions')
plt.show()